In [1]:
import pandas as pd
import os
from IPython.display import display
import sys
from tabulate import tabulate

pd.set_option('mode.chained_assignment', None)

In [2]:
initial_amount = 300
months_to_distribute_initial_amount = 3
term_months = 10
credit_rate_yearly = 0.12
total_time_in_months = 120
total_time_in_months_test = 24

credit_rate_monthly = credit_rate_yearly/12
initial_payment = initial_amount/months_to_distribute_initial_amount

In [3]:
cl_dct = {}
col_names_basic = ["Term","Amount_left"]
col_names_clients = ["Paid","Return_principal","Return_interest"]

cl_data_ini = []
for i in range(total_time_in_months_test+1):
  cl_data_ini.append(("m%02d"%(i+1),None))
df = pd.DataFrame(cl_data_ini,columns=col_names_basic)

amount_left = initial_amount
amount_available = [amount_left]
print("Available amount in the beginning of month:")
print("month_%02d: %0.2f$"%(1,amount_left))


for clt in range(total_time_in_months_test):#-term_months):
  client_number = clt+1
  
  cl_data_ini = []
  if client_number <= total_time_in_months_test-term_months+1:
    for i in range(total_time_in_months_test+1):
      if i+1 < client_number:
        cl_data_ini.append(("m%02d"%(i+1),None,0.0,0.0,0.0))
      elif i+1 == client_number:
        payment = initial_payment if initial_payment <= amount_left else amount_left
        amount_left -= payment
        return_principal = payment/term_months
        principal_left = payment
        cl_data_ini.append(("m%02d"%(i+1),amount_left,payment,0.0,0.0))

      elif i < term_months + client_number:
        cl_data_ini.append(("m%02d"%(i+1),None,0.0,return_principal,principal_left*credit_rate_monthly))
        principal_left -= return_principal
      else:
        cl_data_ini.append(("m%02d"%(i+1),None,0.0,0.0,0.0))

    df0 = pd.DataFrame(cl_data_ini,columns=col_names_basic + list(map(lambda x:x+"_%02d"%client_number,col_names_clients)))  
    df0[col_names_basic[-1]][:client_number-1] = df[col_names_basic[-1]][:client_number-1]
    df = df.drop(col_names_basic[-1],axis=1,).merge(df0,on=col_names_basic[0],sort=True)

    cols = list(df)
    cols.insert(1,cols.pop(cols.index(col_names_basic[-1])))
    df = df[cols]
    
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
  else:
    df[col_names_basic[-1]][client_number-1] = amount_left  
    amount_left += df.loc[client_number,[col for col in df if col.startswith(col_names_clients[1])]].sum()
    
    df[col_names_basic[-1]][client_number] = amount_left
    
  amount_available.append(amount_left)
  print("month_%02d: %0.2f$"%(client_number+1,amount_left))
  
# display(df)
df["Total_return_interest"] = df[[col for col in df if col.startswith(col_names_clients[2])]].sum(axis=1)
df[["Total_cumul_return_interest"]] = df[["Total_return_interest"]].cumsum()
df["Amount_available"] = amount_available

df_result = df[["Term"]+[col for col in df if col.startswith(("Amount","Total_"))]]
df_result = df_result[['Term','Amount_available', 'Amount_left', 'Total_return_interest', 'Total_cumul_return_interest']]


Available amount in the beginning of month:
month_01: 300.00$
month_02: 210.00$
month_03: 130.00$
month_04: 60.00$
month_05: 36.00$
month_06: 39.60$
month_07: 43.56$
month_08: 47.92$
month_09: 52.71$
month_10: 57.98$
month_11: 63.78$
month_12: 60.15$
month_13: 56.17$
month_14: 51.79$
month_15: 50.96$
month_16: 52.46$
month_17: 100.96$
month_18: 145.11$
month_19: 184.46$
month_20: 218.54$
month_21: 246.83$
month_22: 268.74$
month_23: 284.63$
month_24: 294.90$
month_25: 300.00$


In [4]:
print( tabulate(df_result, headers='keys', tablefmt='psql') )

+----+--------+--------------------+---------------+-------------------------+-------------------------------+
|    | Term   |   Amount_available |   Amount_left |   Total_return_interest |   Total_cumul_return_interest |
|----+--------+--------------------+---------------+-------------------------+-------------------------------|
|  0 | m01    |           300      |      200      |               0         |                        0      |
|  1 | m02    |           210      |      110      |               1         |                        1      |
|  2 | m03    |           130      |       30      |               1.9       |                        2.9    |
|  3 | m04    |            60      |        0      |               2.7       |                        5.6    |
|  4 | m05    |            36      |        0      |               3         |                        8.6    |
|  5 | m06    |            39.6    |        0      |               3         |                       11.6    |
|